Problem:
Given various data about a student's personal and academic circumstances, can we predict their final trimester grades?


Dataset explanation / Exploratory Data Analysis (EDA):


Feature engineering and selection:
-Engineered features
-Encoding  
-K Best features selection


Model selection and testing:
A total of 6 regression models were tested: 
- Random Forest
- Gradient Boosting Tree
- K Neighbors 
- SVR
- Ridge Regression
- Stochastic Gradient Descent
Talk about rationale for choosing these 

For each model type, the ideal number of features was chosen by iterating through the feature set choosing the k best features based on mutual information for k = 3 up to the max number of features (49 after one hot encoding categorical features). 
<!-- insert kfeatures.png-->
The performance for each model was evaluated based on the following metrics:
- R^2 
- Mean squared error (MSE)
- Mean absolute error (MAE)
- Relative error: MAE/mean target value (as a proxy for percent error, which was unable to be calculated due to 0 being a true value)

For each model, the best performing feature combination was returned based on relative error: 
<!-- ![Sample Image](figures/relative_e_feature_selection.png)figure out how to resize -->


Results
-Best model
    -Best model was KNeighbors, then RandomForest and GradientBoostingTree
-Best feature set
-Most important features 
    - G1 , G2, Gavg

- Across different runs, the number of ideal features fluctuated but the best performing models were consistent, and G1, G2, Gavg were always selected
    - Implies that many of these features have minimal predictive power compared to 

Regression without grade data 
-Improved performance after scaling age and absence (show data)

Overall limitations and next steps
- Scoring for choosing k best features was based on mutual information 
- Normalizing data 
- Feature selection of encoded features 